# ***"Методи прийняття рішень"***

## Завдання

### **Мета**

Маючи вхідний набір даних, опрацювати його, розв’язати задачу прогнозування ціни оренди нерухомості, описати свої дії у презентації.

Рішення міститиме **Jupyter Notebook** та презентацію. Тут описані вимоги до **Jupyter Notebook**.

### **Вимоги**
Розв’язок має складатися з 3-х основних частин:
1. Аналіз та підготовка даних.
2. Моделювання.
3. Аналіз результатів.


**Частина 1. Аналіз та підготовка даних**

- Проаналізуй діапазони значень, їх середні та дисперсію, розподіли, кореляції, наявність пропущених значень, типи ознак.
- Перевір на наявність пропущених значень. За наявності заповни їх або видали рядки з пропусками.
- За наявності бінарних ознак заміни значення цих ознак на 0 та 1.
- За потреби певні дані можуть бути видалено або модифіковано.

**Частина 2. Моделювання**

- Створи як мінімум 2 різні регресійні моделі для прогнозування цільової змінної “Rent”. 
- Моделі вважаються різними, якщо вони базуються на різних даних або є різними типами моделей.
- Оціни якість моделей як мінімум за 2 метриками.
- Оцінка має бути проведена або на тестовій вибірці, або за допомогою крос-валідації.

**Частина 3. Аналіз результатів**

- Для всіх моделей побудуй графіки порівняння справжніх та спрогнозованих даних.
- Зроби висновки, на яких прикладах кожна модель більше помиляється.
- Зроби порівняння якості моделей. Напиши висновки.

**Додатково**
- Проаналізуй ознаки на мультиколінеарність (Variance inflation factor).
- Знайди оптимальні значення параметрів моделей, наприклад, за допомогою алгоритму GridSearch.
- Проаналізуй важливість ознак (feature importance).
- Побудуй графік залишків (residuals plot).

### **Джерело даних** 

[**Датасет "House Rent Prediction Dataset”:**](https://docs.google.com/spreadsheets/d/1OPnEAT64Patnj_Ifhwn_pM1c15rsBNIoFrtz38A1_W4/edit?gid=881312933#gid=881312933)

https://docs.google.com/spreadsheets/d/1OPnEAT64Patnj_Ifhwn_pM1c15rsBNIoFrtz38A1_W4/edit?gid=881312933#gid=881312933

## **ЧАСТИНА 1: *Аналіз та підготовка даних***

### Отримання даних з посилання на Google Sheets

У зв'язку з тим, що початковий документ, посилання на який дано в завданні, містить різні аркуші, було вирішено почати підготовку даних з пошуку та завантаження саме того аркушу, що містить у назві ключові слова. Для цього було розроблено скрипт на Python, який автоматизує цей процес.

**Функція `find_csv_link_by_keywords`**

Ця функція приймає URL на Google Sheets документ та список ключових слів. Вона здійснює такі кроки:
1. Завантажує HTML-код сторінки за вказаним URL.
2. Парсить HTML-код для пошуку назв аркушів у документі.
3. Перевіряє, чи містять назви аркушів вказані ключові слова.
4. Якщо знаходиться відповідний аркуш, функція формує URL для експорту цього аркуша у форматі `.csv` і повертає цей URL. Якщо аркуш не знайдено, повертається повідомлення про помилку.

**Функція `get_spreadsheet_id`**

Ця допоміжна функція витягує ідентифікатор документа з URL-адреси Google Sheets. Вона використовується в `find_csv_link_by_keywords` для правильного формування URL на `.csv` документ.

**Функція `is_csv_url`**

Перевіряє, чи вказує передана URL-адреса на `.csv` документ. Вона надсилає HTTP-запит типу HEAD до URL і аналізує заголовок `Content-Type`. Якщо заголовок вказує на тип `text/csv`, функція повертає `True`, інакше `False`. Це дозволяє уникнути помилок при завантаженні даних з неправильного формату.


In [4]:
import urllib.parse

import pandas as pd
import requests
from bs4 import BeautifulSoup


def find_csv_link_by_keywords(url: str, keywords: list) -> str:
    """
    Функція приймає URL на Google Sheets документ і список ключових слів.
    Повертає посилання на .csv-документ, що містить дані, пов'язані з цими ключовими словами.

    Parameters
    ----------
    url : str
        URL на Google Sheets документ.
    keywords : list of str
        Список ключових слів для пошуку.

    Returns
    -------
    str
        URL на .csv файл або повідомлення про помилку.
    """
    try:
        # Отримуємо HTML сторінки за допомогою HTTP запиту
        response = requests.get(url)

        # Перевіряємо, чи успішний запит (код 200 означає успіх)
        if response.status_code != 200:
            return f"Помилка доступу до URL: {response.status_code}"

        # Використовуємо BeautifulSoup для парсингу HTML-коду сторінки
        soup = BeautifulSoup(response.text, "html.parser")

        # Збираємо всі назви аркушів (вони зберігаються в <div> з класом 'docs-sheet-tab')
        sheet_titles = [
            div.get_text() for div in soup.find_all("div", class_="docs-sheet-tab")
        ]

        # Перебираємо всі назви аркушів і перевіряємо, чи містять вони ключові слова
        for title in sheet_titles:
            for keyword in keywords:
                # Якщо ключове слово є в назві аркуша (незалежно від регістру)
                if keyword.lower() in title.lower():
                    # Формуємо URL для експорту аркуша у форматі .csv
                    csv_url = (
                        f"https://docs.google.com/spreadsheets/d/"
                        f"{get_spreadsheet_id(url)}/gviz/tq?tqx=out:csv&sheet="
                        f"{urllib.parse.quote(title)}"
                    )
                    return csv_url

        # Якщо жоден аркуш не відповідає ключовим словам, повертаємо повідомлення
        return "Не знайдено аркушів, що відповідають ключовим словам."

    except Exception as e:
        # Якщо виникла будь-яка інша помилка, повертаємо її опис
        return f"Сталася помилка: {str(e)}"


def get_spreadsheet_id(url: str) -> str:
    """
    Функція для витягування ID документа з URL.

    Parameters
    ----------
    url : str
        URL на Google Sheets документ.

    Returns
    -------
    str
        ID документа.

    Raises
    ------
    ValueError
        Якщо не вдалося витягти ID документа з URL.
    """
    # Парсимо URL для отримання окремих частин
    parsed_url = urllib.parse.urlparse(url)
    path_parts = parsed_url.path.split("/")

    # Ідентифікатор документа знаходиться між 'd' і наступним елементом у URL
    if "d" in path_parts:
        return path_parts[path_parts.index("d") + 1]
    else:
        # Якщо структура URL не відповідає очікуваній, генеруємо помилку
        raise ValueError("Не вдалося витягти ID документа з URL.")


def is_csv_url(url: str) -> bool:
    """
    Функція перевіряє, чи вказує передана URL-адреса на .csv-документ.

    Parameters
    ----------
    url : str
        URL для перевірки.

    Returns
    -------
    bool
        True, якщо URL вказує на .csv документ, інакше False.
    """
    try:
        # Здійснюємо запит до URL
        response = requests.head(url, allow_redirects=True)

        # Перевіряємо заголовки, зокрема Content-Type
        content_type = response.headers.get("Content-Type", "")

        # Повертаємо True, якщо Content-Type вказує на CSV
        return "text/csv" in content_type
    except Exception as e:
        print(f"Помилка при перевірці URL: {str(e)}")
        return False


# Тестування функції
keywords = ["house", "rent"]
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1OPnEAT64Patnj_Ifhwn_pM1c15rsBNIoFrtz38A1_W4/edit?gid=881312933#gid=881312933"
csv_link = find_csv_link_by_keywords(spreadsheet_url, keywords)
print(f'Результат роботи функції "find_csv_link_by_keywords": {csv_link}')

print(f'Перевірка отриманого посилання: {csv_link.startswith("http")}')
# Перевірка, чи URL вказує на .csv файл
if is_csv_url(csv_link):
    # Завантажуємо дані у DataFrame за допомогою pandas
    df = pd.read_csv(csv_link)
    # Виводимо перші 5 рядків таблиці для перевірки
    print(df.head())
else:
    print("Помилка: не вдалося завантажити дані.")



Результат роботи функції "find_csv_link_by_keywords": https://docs.google.com/spreadsheets/d/1OPnEAT64Patnj_Ifhwn_pM1c15rsBNIoFrtz38A1_W4/gviz/tq?tqx=out:csv&sheet=House_Rent_Dataset
Перевірка отриманого посилання: True
    Posted On  BHK   Rent  Size            Floor    Area Type  \
0  2022-05-18    2  10000  1100  Ground out of 2   Super Area   
1  2022-05-13    2  20000   800       1 out of 3   Super Area   
2  2022-05-16    2  17000  1000       1 out of 3   Super Area   
3  2022-07-04    2  10000   800       1 out of 2   Super Area   
4  2022-05-09    2   7500   850       1 out of 2  Carpet Area   

              Area Locality     City Furnishing Status  Tenant Preferred  \
0                    Bandel  Kolkata       Unfurnished  Bachelors/Family   
1  Phool Bagan, Kankurgachi  Kolkata    Semi-Furnished  Bachelors/Family   
2   Salt Lake City Sector 2  Kolkata    Semi-Furnished  Bachelors/Family   
3               Dumdum Park  Kolkata       Unfurnished  Bachelors/Family   
4        

### Аналіз та підготовка даних

#### **Завантаження та попередній огляд даних**

Спершу завантажимо набір даних і подивимося на перші кілька рядків, щоб зрозуміти структуру та зміст даних.
Для завантаження скористаємось функцією `find_csv_link_by_keywords`, створеною раніше.

In [5]:
keywords = ["house", "rent"]
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1OPnEAT64Patnj_Ifhwn_pM1c15rsBNIoFrtz38A1_W4/edit?gid=881312933#gid=881312933"
csv_link = find_csv_link_by_keywords(spreadsheet_url, keywords)

# Виведення перших 5 рядків (за замовчуванням) набору даних
print("Перші кілька рядків даних:")
print(df.head())


Перші кілька рядків даних:
    Posted On  BHK   Rent  Size            Floor    Area Type  \
0  2022-05-18    2  10000  1100  Ground out of 2   Super Area   
1  2022-05-13    2  20000   800       1 out of 3   Super Area   
2  2022-05-16    2  17000  1000       1 out of 3   Super Area   
3  2022-07-04    2  10000   800       1 out of 2   Super Area   
4  2022-05-09    2   7500   850       1 out of 2  Carpet Area   

              Area Locality     City Furnishing Status  Tenant Preferred  \
0                    Bandel  Kolkata       Unfurnished  Bachelors/Family   
1  Phool Bagan, Kankurgachi  Kolkata    Semi-Furnished  Bachelors/Family   
2   Salt Lake City Sector 2  Kolkata    Semi-Furnished  Bachelors/Family   
3               Dumdum Park  Kolkata       Unfurnished  Bachelors/Family   
4             South Dum Dum  Kolkata       Unfurnished         Bachelors   

   Bathroom Point of Contact  
0         2    Contact Owner  
1         1    Contact Owner  
2         1    Contact Owner  
3